In [3]:
%pwd

'c:\\Users\\sanoj\\OneDrive\\Desktop\\new'

In [1]:
import os##load pdf


os.chdir("../") #folder back prevent path working on project directory

In [2]:
%pwd

'c:\\Users\\sanoj\\OneDrive\\Desktop\\new'

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
##load pdf

def load_Pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [9]:
extracted_data = load_Pdf_file(data='data/')

In [6]:
##split the data into text chunks

def text_splits(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks 





In [7]:
text_chunks = text_splits(extracted_data)
print(len(text_chunks))

NameError: name 'extracted_data' is not defined

In [8]:
##embedding

from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\sanoj\AppData\Local\Temp\ipykernel_13216\3245925232.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\sanoj\OneDrive\Desktop\medical\end_to_end_medical_chatbot1\mychatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINCONE_API_KEY')

In [12]:
##pincone
from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot2"
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,  # Match this to your embedding model's output dimension
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'df90f75c57727f9ece804a070b82e1d2', 'date': 'Tue, 24 Jun 2025 07:23:56 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")



os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [14]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
    
)

In [15]:
##exisitng index
from langchain_pinecone import PineconeVectorStore

##embed each chunks
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
#semantic search 

retriver = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [27]:
retrieved_doc=retriver.invoke("what is ache")
print(retrieved_doc)

[Document(id='2750cb5d-5284-48af-a0d8-98832728a77f', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 503.0, 'page_label': '504', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Gale Encyclopedia of Medicine Vol. 2 (C-F).pdf', 'total_pages': 759.0}, page_content='dysmenorrhea (cramps), a dull or throbbing pain that\nGALE ENCYCLOPEDIA OF MEDICINE 21116\nDysmenorrhea'), Document(id='be0962ec-a7a9-4364-8eef-8fe5f209981e', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 531.0, 'page_label': '532', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Gale Encyclopedia of Medicine Vol. 2 (C-F).pdf', 'total_pages': 759.0}, page_content='determine the settings and electrode placements for the\nbest pain relief.\nKEY TERMS\nFibromyalgia —A condition characterized by\naching and stiffness, fatigue and poor sleep, as\nwell as tenderness 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")




os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [ ]:
from langchain_groq import ChatGroq


groq_llm = ChatGroq(
    temperature=0,
    model="llama-3.3-70b-versatile",
)

In [53]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")




from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=2)


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [54]:
from googletrans import Translator
Translator=Translator()



In [ ]:
def translate_to_eng(text:str) ->str:
    result = Translator.translate(text,desc="en")
    return result.text

def translate_back(text:str, target_lang:str) -> str:
    result=Translator.translate(text,desc=target_lang)
    return result.text